In [3]:
import pandas as pd
import sqlite3
import json
from pathlib import Path

from helper import load_dotenv
from get_labelstudio_data import get_moires_of_project, get_tasks, get_results_of_project

In [5]:
config = load_dotenv()

In [ ]:
config

In [ ]:
moires = get_moires_of_project( config["LABEL_STUDIO_PROJECT_ID"], config["LABEL_STUDIO_TOKEN"] )

In [ ]:
len(moires)

In [ ]:
con = sqlite3.connect( config["LABEL_STUDIO_DB_PATH"] )

In [ ]:
def get_results_of_project( project_id ):
    # tasks by id
    tasks = get_tasks( config["LABEL_STUDIO_PROJECT_ID"] )

    tasks_by_id = {}
    
    for t in tasks:
        tasks_by_id[t['id']] = Path(t['data_path']).name

    # results
    c = con.cursor()
    c.execute(
        f"""
            SELECT task_id,result FROM task_completion
            WHERE result != "[]" AND task_id IN ({ ','.join([str(t['id']) for t in tasks]) })
        """
    )
    rows = c.fetchall()
    c.close()
    
    results = []

    for r in rows:
        task_id = int(r[0])
        res_task = json.loads(r[1])

        for res in res_task:
            res['task_id'] = task_id
            res['img_name'] = tasks_by_id[task_id]
            results.append(res)

    return results

In [14]:
m1 = get_results_of_project( config["LABEL_STUDIO_PROJECT_ID"], labels=['potential_moire'] )

In [15]:
len(m1)

11

In [ ]:
assert len(m1) == len(moires)

In [ ]:
for i in range(len(m1)):
    assert m1[i] == moires[i], f'position {i} not equal'